go/meterstick-demo-pssession

#Summary

Fully supported:

*   Raw column name, like cpc_cost_usd.
*   Raw table name, like shopping_analysis.ps_session.all.
*   Complete query, like SELECT * FROM shopping_analysis.ps_session.all, UNNEST(ps_event).

Partially supported:

*   table.column, like ev.cpc_cost_usd.

Preaggregating data can save memory.


In [1]:
from colabtools import adhoc_import

with adhoc_import.Google3():
  from meterstick.v2 import *

In [2]:
%%dremel
SET SQL_DIALECT GOOGLESQL;
SET MIN_COMPLETION_RATIO 1;

Done! Processed commands DEFINE: 0, LOAD: 0, SET: 2


## MH Metrics

In [ ]:
table = '''shopping_analysis.ps_session.all AS t,
UNNEST(t.ps_event) AS ev,
UNNEST(ev.experimentid) AS expt_id,
UNNEST(ev.impression_set) AS imp'''

filters = '''
  _PARTITION_DATE BETWEEN '20200904' AND '20200905'
  AND ev.page_type != 'AFS'
  AND ev.good_for_revenue
  AND imp.slot > 0
  AND imp.impression_type = 'PLA'
  AND imp.clicks_valid_for_revenue > 0
  AND conversion_within_threshold
  AND expt_id IN (2004375, 2004376)'''

spend = 'cpc_cost_usd'
conversions = 'conversions_valid_for_revenue'
clicks = 'clicks_valid_for_revenue'
cvr = Ratio(conversions, clicks, 'cvr')
cpc = Ratio(spend, clicks, 'cpc')
cpd = Ratio(conversions, spend, 'cpd')
ms = MetricList((cpc, cvr, cpd), where=filters)
mh = MH('expt_id', 2004375, 'ads_customer_id', ms)
jk = Jackknife('rasta_cookie_bucket', mh, 0.95)
res = jk.compute_on_sql(table)
res

Metric  cpc_mh_ratio                     ...       cpd_mh_ratio                   
               Value Jackknife CI-lower  ... Jackknife CI-lower Jackknife CI-upper
expt_id                                  ...                                      
2004376     0.063509          -0.282558  ...           -0.55014           1.133661

[1 rows x 9 columns]

In [ ]:
res.display(metric_formats={'Ratio': '{:.3f}%'})

Agrees to [SEAT](https://gearloose.corp.google.com/#/seat/FK0AQR/results).

![SEAT](https://screenshot.googleplex.com/7eixS6HXD9N4KVU.png)

But SEAT doesn't support MH with slicing.

In [ ]:
res = jk.compute_on_sql(table, 'ux_interface')
res

Metric               cpc_mh_ratio  ...       cpd_mh_ratio
                            Value  ... Jackknife CI-upper
ux_interface expt_id               ...                   
1            2004376    -0.053896  ...           2.032925
2            2004376     0.064048  ...           1.391591
3            2004376     0.776793  ...           6.674100

[3 rows x 9 columns]

In [ ]:
res.display()

Column name as 'table.column' may or may not work, but it won't give you a wrong result without notice. The SQL just won't compile.

In [ ]:
# table.column style still works for MH.
spend_with_tbl = 'imp.cpc_cost_usd'
conversions_with_tbl = 'imp.conversions_valid_for_revenue'
clicks_with_tbl = 'imp.clicks_valid_for_revenue'
cvr_with_tbl = Ratio(conversions_with_tbl, clicks_with_tbl, 'cvr')
cpc_with_tbl = Ratio(spend_with_tbl, clicks_with_tbl, 'cpc')
cpd_with_tbl = Ratio(conversions_with_tbl, spend_with_tbl, 'cpd')
ms_with_tbl = MetricList((cvr_with_tbl, cpc_with_tbl, cpd_with_tbl),
                         where=filters)
mh_with_tbl = MH('expt_id', 2004375, 'ads_customer_id', ms_with_tbl)
mh_with_tbl.compute_on_sql(table)

,cvr_mh_ratio,cpc_mh_ratio,cpd_mh_ratio
expt_id,,,
2004376,0.421622,-0.03072,0.590635


In [ ]:
# But fails on Jackknife.
jk_with_tbl = Jackknife('rasta_cookie_bucket', mh_with_tbl, 0.95)
jk_with_tbl.compute_on_sql(table)

GoogleSqlExecutionError: ignored

## Difference-in-Differences

In [ ]:
# Query compiles but exhausts memory.
pre = PercentChange('expt_id', 2004375, ms, where='date = "2020-09-04"')
post = PercentChange('expt_id', 2004375, ms, where='date = "2020-09-05"')
did = post - pre
jk = Jackknife('rasta_cookie_bucket', did, 0.95)
res = jk.compute_on_sql(table)
res

GoogleSqlExecutionError: ignored

In [ ]:
# Preaggregation solves the memory issue.
table = '''
SELECT 
  date,
  expt_id,
  rasta_cookie_bucket,
  SUM(cpc_cost_usd) AS cpc_cost_usd,
  SUM(conversions_valid_for_revenue) AS conversions_valid_for_revenue,
  SUM(clicks_valid_for_revenue) AS clicks_valid_for_revenue,
FROM shopping_analysis.ps_session.all AS t,
  UNNEST(t.ps_event) AS ev,
  UNNEST(ev.experimentid) AS expt_id,
  UNNEST(ev.impression_set) AS imp
WHERE
  _PARTITION_DATE BETWEEN '20200904' AND '20200905'
  AND ev.page_type != 'AFS'
  AND ev.good_for_revenue
  AND imp.slot > 0
  AND imp.impression_type = 'PLA'
  AND imp.clicks_valid_for_revenue > 0
  AND conversion_within_threshold
  AND expt_id IN (2004375, 2004376)
GROUP BY 1, 2, 3
'''

ms = MetricList((cpc, cvr, cpd))
pre = PercentChange('expt_id', 2004375, ms, where='date = "2020-09-04"')
post = PercentChange('expt_id', 2004375, ms, where='date = "2020-09-05"')
did = post - pre
jk = Jackknife('rasta_cookie_bucket', did, 0.95)
res = jk.compute_on_sql(table)
res

Metric  cpc_percent_change_minus_cpc_percent_change  ... cpd_percent_change_minus_cpd_percent_change
                                              Value  ...                          Jackknife CI-upper
expt_id                                              ...                                            
2004376                                    0.123881  ...                                    2.353186

[1 rows x 9 columns]

In [ ]:
res.display()